## Bayesian methods of hyperparameter optimization

In addition to the random search and the grid search methods for selecting optimal hyperparameters, we can use Bayesian methods of probabilities to select the optimal hyperparameters for an algorithm.

In this case study, we will be using the BayesianOptimization library to perform hyperparmater tuning. This library has very good documentation which you can find here: https://github.com/fmfn/BayesianOptimization

You will need to install the Bayesian optimization module. Running a cell with an exclamation point in the beginning of the command will run it as a shell command — please do this to install this module from our notebook in the cell below.

In [2]:
! pip install bayesian-optimization

In [3]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import lightgbm
from bayes_opt import BayesianOptimization
from catboost import CatBoostClassifier, cv, Pool

In [4]:
import os
os.listdir()

['.DS_Store',
 '.ipynb_checkpoints',
 'Bayesian_optimization_case_study.ipynb',
 'flight_delays_test.csv',
 'flight_delays_test.csv.zip',
 'flight_delays_train.csv',
 'flight_delays_train.csv.zip']

## How does Bayesian optimization work?

Bayesian optimization works by constructing a posterior distribution of functions (Gaussian process) that best describes the function you want to optimize. As the number of observations grows, the posterior distribution improves, and the algorithm becomes more certain of which regions in parameter space are worth exploring and which are not, as seen in the picture below.

<img src="https://github.com/fmfn/BayesianOptimization/blob/master/examples/bo_example.png?raw=true" />
As you iterate over and over, the algorithm balances its needs of exploration and exploitation while taking into account what it knows about the target function. At each step, a Gaussian Process is fitted to the known samples (points previously explored), and the posterior distribution, combined with an exploration strategy (such as UCB — aka Upper Confidence Bound), or EI (Expected Improvement). This process is used to determine the next point that should be explored (see the gif below).
<img src="https://github.com/fmfn/BayesianOptimization/raw/master/examples/bayesian_optimization.gif" />

## Let's look at a simple example

The first step is to create an optimizer. It uses two items:
* function to optimize
* bounds of parameters

The function is the procedure that counts metrics of our model quality. The important thing is that our optimization will maximize the value on function. Smaller metrics are best. Hint: don't forget to use negative metric values.

Here we define our simple function we want to optimize.

In [5]:
def simple_func(a, b):
    return a + b

Now, we define our bounds of the parameters to optimize, within the Bayesian optimizer.

In [6]:
optimizer = BayesianOptimization(
    simple_func,
    {'a': (1, 3),
    'b': (4, 7)})

These are the main parameters of this function:

* **n_iter:** This is how many steps of Bayesian optimization you want to perform. The more steps, the more likely you are to find a good maximum.

* **init_points:** This is how many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.

Let's run an example where we use the optimizer to find the best values to maximize the target value for a and b given the inputs of 3 and 2.

In [7]:
optimizer.maximize(3,2)

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
| 1         | 7.341     | 2.587     | 4.753     |
| 2         | 8.643     | 2.126     | 6.518     |
| 3         | 7.338     | 1.957     | 5.381     |
| 4         | 9.147     | 2.181     | 6.966     |
| 5         | 8.0       | 1.0       | 7.0       |


Great, now let's print the best parameters and the associated maximized target.

In [49]:
print(optimizer.max['params']);optimizer.max['target']

{'a': 3.0, 'b': 7.0}


10.0

## Test it on real data using the Light GBM

The dataset we will be working with is the famous flight departures dataset. Our modeling goal will be to predict if a flight departure is going to be delayed by 15 minutes based on the other attributes in our dataset. As part of this modeling exercise, we will use Bayesian hyperparameter optimization to identify the best parameters for our model.

**<font color='teal'> You can load the zipped csv files just as you would regular csv files using Pandas read_csv. In the next cell load the train and test data into two seperate dataframes. </font>**


In [8]:
train_df = pd.read_csv('flight_delays_train.csv.zip')
test_df = pd.read_csv('flight_delays_test.csv.zip')

**<font color='teal'> Print the top five rows of the train dataframe and review the columns in the data. </font>**

In [9]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


**<font color='teal'> Use the describe function to review the numeric columns in the train dataframe. </font>**

In [10]:
train_df.describe()

,DepTime,Distance
count,100000.000000,100000.00000
mean,1341.523880,729.39716
std,476.378445,574.61686
min,1.000000,30.00000
25%,931.000000,317.00000
50%,1330.000000,575.00000
75%,1733.000000,957.00000
max,2534.000000,4962.00000


Notice, `DepTime` is the departure time in a numeric representation in 2400 hours. 

 **<font color='teal'>The response variable is 'dep_delayed_15min' which is a categorical column, so we need to map the Y for yes and N for no values to 1 and 0. Run the code in the next cell to do this.</font>**

In [11]:
#train_df = train_df[train_df.DepTime <= 2400].copy()
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

## Feature Engineering
Use these defined functions to create additional features for the model. Run the cell to add the functions to your workspace.

In [12]:
def label_enc(df_column):
    df_column = LabelEncoder().fit_transform(df_column)
    return df_column

def make_harmonic_features_sin(value, period=2400):
    value *= 2 * np.pi / period 
    return np.sin(value)

def make_harmonic_features_cos(value, period=2400):
    value *= 2 * np.pi / period 
    return np.cos(value)

def feature_eng(df):
    df['flight'] = df['Origin']+df['Dest']
    df['Month'] = df.Month.map(lambda x: x.split('-')[-1]).astype('int32')
    df['DayofMonth'] = df.DayofMonth.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['begin_of_month'] = (df['DayofMonth'] < 10).astype('uint8')
    df['midddle_of_month'] = ((df['DayofMonth'] >= 10)&(df['DayofMonth'] < 20)).astype('uint8')
    df['end_of_month'] = (df['DayofMonth'] >= 20).astype('uint8')
    df['DayOfWeek'] = df.DayOfWeek.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['hour'] = df.DepTime.map(lambda x: x/100).astype('int32')
    df['morning'] = df['hour'].map(lambda x: 1 if (x <= 11)& (x >= 7) else 0).astype('uint8')
    df['day'] = df['hour'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8')
    df['evening'] = df['hour'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8')
    df['night'] = df['hour'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32')
    df['winter'] = df['Month'].map(lambda x: x in [12, 1, 2]).astype('int32')
    df['spring'] = df['Month'].map(lambda x: x in [3, 4, 5]).astype('int32')
    df['summer'] = df['Month'].map(lambda x: x in [6, 7, 8]).astype('int32')
    df['autumn'] = df['Month'].map(lambda x: x in [9, 10, 11]).astype('int32')
    df['holiday'] = (df['DayOfWeek'] >= 5).astype(int) 
    df['weekday'] = (df['DayOfWeek'] < 5).astype(int)
    df['airport_dest_per_month'] = df.groupby(['Dest', 'Month'])['Dest'].transform('count')
    df['airport_origin_per_month'] = df.groupby(['Origin', 'Month'])['Origin'].transform('count')
    df['airport_dest_count'] = df.groupby(['Dest'])['Dest'].transform('count')
    df['airport_origin_count'] = df.groupby(['Origin'])['Origin'].transform('count')
    df['carrier_count'] = df.groupby(['UniqueCarrier'])['Dest'].transform('count')
    df['carrier_count_per month'] = df.groupby(['UniqueCarrier', 'Month'])['Dest'].transform('count')
    df['deptime_cos'] = df['DepTime'].map(make_harmonic_features_cos)
    df['deptime_sin'] = df['DepTime'].map(make_harmonic_features_sin)
    df['flightUC'] = df['flight']+df['UniqueCarrier']
    df['DestUC'] = df['Dest']+df['UniqueCarrier']
    df['OriginUC'] = df['Origin']+df['UniqueCarrier']
    return df.drop('DepTime', axis=1)

Concatenate the training and testing dataframes.


In [13]:
full_df = pd.concat([train_df.drop('dep_delayed_15min', axis=1), test_df])
full_df = feature_eng(full_df)

Apply the earlier defined feature engineering functions to the full dataframe.

In [14]:
for column in ['UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']:
    full_df[column] = label_enc(full_df[column])


Split the new full dataframe into X_train and X_test. 

In [15]:
X_train = full_df[:train_df.shape[0]]
X_test = full_df[train_df.shape[0]:]

Create a list of the categorical features.

In [16]:
categorical_features = ['Month',  'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']

Let's build a light GBM model to test the bayesian optimizer.

### [LightGBM](https://lightgbm.readthedocs.io/en/latest/) is a gradient boosting framework that uses tree-based learning algorithms. It is designed to be distributed and efficient with the following advantages:

* Faster training speed and higher efficiency.
* Lower memory usage.
* Better accuracy.
* Support of parallel and GPU learning.
* Capable of handling large-scale data.

First, we define the function we want to maximize and that will count cross-validation metrics of lightGBM for our parameters.

Some params such as num_leaves, max_depth, min_child_samples, min_data_in_leaf should be integers.

In [34]:
# This parameter is no longer supported: early_stopping_rounds=100 
# In the code you get this error: TypeError: cv() got an unexpected keyword argument 'early_stopping_rounds'
# 'auc-mean' does not exist it has been renamed to 'valid auc-mean'

def lgb_eval(num_leaves,max_depth,lambda_l2,lambda_l1,min_child_samples, min_data_in_leaf):
    params = {
        "objective" : "binary",
        "metric" : "auc", 
        'is_unbalance': True,
        "num_leaves" : int(num_leaves),
        "max_depth" : int(max_depth),
        "lambda_l2" : lambda_l2,
        "lambda_l1" : lambda_l1,
        "num_threads" : 20,
        "min_child_samples" : int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        "learning_rate" : 0.03,
        "subsample_freq" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1
    }
    lgtrain = lightgbm.Dataset(X_train, y_train,categorical_feature=categorical_features)
    cv_result = lightgbm.cv(params,
                       lgtrain,
                       1000,
                       stratified=True,
                       nfold=3)
    
    return cv_result['valid auc-mean'][-1]

Apply the Bayesian optimizer to the function we created in the previous step to identify the best hyperparameters. We will run 10 iterations and set init_points = 2.


In [35]:
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (25, 4000),
                                                'max_depth': (5, 63),
                                                'lambda_l2': (0.0, 0.05),
                                                'lambda_l1': (0.0, 0.05),
                                                'min_child_samples': (50, 10000),
                                                'min_data_in_leaf': (100, 2000)
                                                })

lgbBO.maximize(n_iter=10, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
{'valid auc-mean': [0.6994417494764628, 0.7054642054125151, 0.7078419168265837, 0.7106352635846681, 0.7118970232534299, 0.7133934147891479, 0.7141138595027715, 0.7149326328550222, 0.715537253195043, 0.7160631073385199, 0.7163788908985178, 0.7168075393471035, 0.7171175923234466, 0.7174289858302959, 0.717638117446783, 0.7180449437175181, 0.7182323918314953, 0.7183888608794889, 0.7188330759821548, 0.7190293903128989, 0.7193651717830433, 0.7196224682295672, 0.7197656910786935, 0.7198800532284682, 0.7201473162432185, 0.7203461557066708, 0.7205287454467921, 0.7205921594811692, 0.7208503678627087, 0.7210096722440676, 0.721145126498867, 0.7211544623122125, 0.7213296793894872, 0.7214441747091174, 0.7215670774573204, 0.7216950786032866, 0.7218414183334095, 0.7219942028797814, 0.7221549122285614, 0.72225

{'valid auc-mean': [0.6989018748916539, 0.7053839078110901, 0.7087243134856894, 0.7111789905691875, 0.7129031198456709, 0.7138067895024327, 0.7150161506337795, 0.7159912037968513, 0.7165126786140464, 0.7168694020669052, 0.7171706035071149, 0.7177188998207988, 0.7182777364456641, 0.7186924048810389, 0.7189206050514306, 0.7192384371862569, 0.7196440724930632, 0.7199018541829324, 0.7199118161574737, 0.7202534649492272, 0.7205022730490311, 0.7206882013588304, 0.7207124065544449, 0.7208751996474412, 0.7210338390922248, 0.7212068256837009, 0.7212659287587542, 0.7213519146081068, 0.7214579299124342, 0.7216978252826965, 0.721839015080097, 0.7219582362283955, 0.721971768661958, 0.7221030477254695, 0.7222864728292122, 0.7223437556705111, 0.7224936662448181, 0.7225330198253678, 0.7226270092331298, 0.722688136877895, 0.7228143816283129, 0.7228742613550173, 0.7229342095151273, 0.7229535676446514, 0.7229851403296464, 0.7230309427656954, 0.7231461486005579, 0.7232043467984131, 0.7233279928658197, 0.7

{'valid auc-mean': [0.6990203618736878, 0.7060765990369559, 0.7099626600640786, 0.7115997793606651, 0.7140078082305368, 0.7150313987311603, 0.7164174946595462, 0.717544542820142, 0.7182457603077559, 0.7185526833373542, 0.718863134706822, 0.7191738597887477, 0.7195047022783218, 0.7197438532166162, 0.7199772897556208, 0.719899725517752, 0.7202489180877375, 0.7203218227548214, 0.7204081083752999, 0.7205040735556313, 0.7205554188857355, 0.720709112101335, 0.7207137124292573, 0.72080020383931, 0.7208563871071544, 0.7208461389566493, 0.7208910242661176, 0.721101120349647, 0.7211643778806621, 0.7212596665181631, 0.7214371271530419, 0.7214477594601124, 0.7214372136264645, 0.7214423576047236, 0.7215260096374738, 0.7215778341757719, 0.721629069784683, 0.7217177909927726, 0.721797010360763, 0.7217171308103939, 0.7217997210911579, 0.7217135082130172, 0.7216586323376694, 0.7217742519120591, 0.7216789192929035, 0.7216671910063641, 0.7216037662372964, 0.7216384533222825, 0.7216137314545152, 0.7215793

{'valid auc-mean': [0.6942898554774732, 0.7023851925106638, 0.7070377166084719, 0.7091508842084456, 0.7114087270956905, 0.7124488786368882, 0.7139047425680355, 0.714463675760194, 0.7155161675213648, 0.7159793462219352, 0.7164734272153382, 0.7168568725517851, 0.7174012302447123, 0.7176044468386987, 0.7177348422329671, 0.7179379287243023, 0.7180049500611739, 0.7180273700253488, 0.7181448666192177, 0.7182985756276379, 0.7183290264076065, 0.7184522989288249, 0.7186049985654629, 0.7186524460075502, 0.7186762819544796, 0.7188034231757587, 0.7188261199969667, 0.7188020489717722, 0.7187926136002556, 0.7185850461456423, 0.7185434628056694, 0.7185918527211593, 0.7185644604843829, 0.7184643280828139, 0.7184854047047922, 0.7183549741774778, 0.7183021155058369, 0.7182866031631819, 0.7181642027950499, 0.7180435789867365, 0.7180599313992219, 0.7180091526075882, 0.7179600300168381, 0.718003120566752, 0.7180287245184411, 0.7179440253945808, 0.717916922342673, 0.7178073002962707, 0.7177811349149689, 0.7

{'valid auc-mean': [0.6963047663437002, 0.7027082821699967, 0.7055692775227246, 0.7072584439216766, 0.7086457633121542, 0.7095965651212817, 0.7102654995206151, 0.7111003288970076, 0.7120841733436641, 0.712617809728295, 0.7129933233153097, 0.7134221908662947, 0.7139800932552184, 0.7141907371042997, 0.7143672832469399, 0.714755480230461, 0.7151446719764089, 0.7155993717018282, 0.7158145607245413, 0.7161104127409069, 0.7161644254815905, 0.7163625212911882, 0.7168677667414171, 0.7169260113055964, 0.7171286659644597, 0.7173287476707463, 0.7176572336058199, 0.7179681480014238, 0.7181515658308464, 0.7182898840583373, 0.7184927695454811, 0.7186095722933338, 0.7187775046122313, 0.7189695124317881, 0.7190200019398015, 0.7193066495741275, 0.7195655574008727, 0.7197672269273477, 0.7198848364161886, 0.719988285051213, 0.7200951558957266, 0.7202481429428462, 0.7203973273299521, 0.7205262890283485, 0.7207258869114482, 0.7208293728582488, 0.7209329729034968, 0.7210512582568519, 0.7211782821273794, 0.7

...

{'valid auc-mean': [0.699316791839007, 0.7052260888150891, 0.7081310128471824, 0.710466600464586, 0.7118783827753726, 0.7130724310229545, 0.7138514484041031, 0.7147564804852137, 0.7154380399762742, 0.7163113360349244, 0.7164774690984684, 0.7169202373473245, 0.7172294712716086, 0.717613986529614, 0.7180379979131265, 0.7183771425702221, 0.7184244384618005, 0.718632744642329, 0.7189266648135552, 0.7190637116154113, 0.7192536860630483, 0.7193993201785496, 0.7197375825375282, 0.71994289829215, 0.7202624428280707, 0.7204732353294774, 0.7206888222622302, 0.7208181541840619, 0.7209909050302777, 0.7210563068730237, 0.7211783811047604, 0.7213471165270716, 0.7214110852080461, 0.7214846429252404, 0.7216153490163384, 0.7217036183313406, 0.7218815975985087, 0.721994649056721, 0.7221715773120736, 0.7223710488893822, 0.7224043665859062, 0.7224566239146251, 0.7225721933985868, 0.7226092098156971, 0.7227305415199714, 0.7227586585580772, 0.7228834565938166, 0.7229240059117447, 0.7229980333450717, 0.72326

{'valid auc-mean': [0.6974409287349581, 0.7036684668518198, 0.7068482139411914, 0.7085983931546412, 0.7102206857384719, 0.7110776053458462, 0.7121785591100357, 0.7129779507910811, 0.7139610553901893, 0.7142012307844495, 0.714748102626082, 0.7151397639800668, 0.7156583232846717, 0.7161801718494875, 0.7167150157031826, 0.7170475986077882, 0.7172610867661499, 0.717551168068724, 0.7179490315809248, 0.7181498349172498, 0.7184598241258359, 0.7187006001037256, 0.7189714682416275, 0.719253539850483, 0.7195847200400557, 0.7197638666222836, 0.7199368751324432, 0.7200768982455806, 0.7200677185263146, 0.7202242363945972, 0.7204215261340231, 0.7206112339971082, 0.7208351510239256, 0.72090923358919, 0.7210403737838601, 0.7211543102162459, 0.7212871213237997, 0.7214186142387283, 0.7214914483728506, 0.7216696861032389, 0.7217741250926819, 0.7219336405152817, 0.7220708978690928, 0.722222646690888, 0.7222614604628483, 0.7223886672290115, 0.7225427495646293, 0.7226875541872916, 0.7228179913054906, 0.7230

{'valid auc-mean': [0.6975167062797666, 0.704265934339778, 0.707454837790609, 0.708638477988634, 0.7101163702280228, 0.7110099116867445, 0.7121737353311949, 0.7129587830111689, 0.713429850430904, 0.7141764181795387, 0.714579604598636, 0.7148687653432823, 0.71533855474264, 0.7157181611047251, 0.7162524654008734, 0.7166338242829049, 0.7170480664941552, 0.7173593283933957, 0.7176488589604869, 0.718052249647256, 0.7182421944745075, 0.7185378999140563, 0.718732246839342, 0.7190616708518163, 0.7192516207861589, 0.7195239063863546, 0.719677568455015, 0.7199336143125054, 0.720054088233363, 0.7202519283178092, 0.7203145579295738, 0.7204341813629312, 0.7206039034757309, 0.7207657394248943, 0.7209926405569926, 0.7211083879754446, 0.7212923697036238, 0.7213783773213519, 0.7215122502794987, 0.7217023581050577, 0.7218936624933742, 0.7220352029246008, 0.7222154996396993, 0.7223357132638214, 0.7224618467366763, 0.7225951744726496, 0.7227710291991908, 0.7229047703250778, 0.723063105355152, 0.7232226392

{'valid auc-mean': [0.6974351902905681, 0.7036544855961594, 0.706830280296734, 0.708583528325533, 0.7102030779835137, 0.7110698595803807, 0.7119707299936183, 0.7129394386349027, 0.7135609126129295, 0.7140122149823217, 0.7146593044506918, 0.714885703913087, 0.715276137504412, 0.7157360732573395, 0.7162059486586445, 0.7166781791003349, 0.7170833800832447, 0.7173682183586482, 0.7176263750403905, 0.7179764759752733, 0.7180111525783417, 0.7184234039095846, 0.7185607079778554, 0.7188509561416838, 0.7189913342890483, 0.7191392245229687, 0.7193721116094286, 0.7196164656225021, 0.7198508742332663, 0.7200194646648388, 0.7202311666162897, 0.7203857241925622, 0.7205663042250278, 0.7206892953886187, 0.7207859337859986, 0.720969887576242, 0.7211718207386825, 0.7212471401251568, 0.7213659414604511, 0.7215813904040749, 0.7217152381707918, 0.7218302498444, 0.7220021175210786, 0.7220804316690353, 0.7222105478495789, 0.72236727831504, 0.7225763983718897, 0.7227313209292685, 0.7228684892294316, 0.72302417

{'valid auc-mean': [0.6990226419580688, 0.7059301927220497, 0.710135639850907, 0.7120984614346854, 0.7142192516297147, 0.7153472820324828, 0.7164979357816166, 0.7172677304558474, 0.7178883865731768, 0.7187271245187373, 0.7192084019759059, 0.7191113311966307, 0.7193163753891145, 0.7197294461682509, 0.7198605106348607, 0.7200644045230744, 0.7201649812221712, 0.7204845371658709, 0.7206646644429869, 0.7209057179495918, 0.7210304317198067, 0.7212305642497832, 0.7213181146376272, 0.7215512731322754, 0.7216876397117957, 0.7217836057824755, 0.7217572708473138, 0.7217821200323252, 0.7218582984563112, 0.7221075947541045, 0.7222431574376049, 0.7222486172157826, 0.7224092742098848, 0.7224460500153751, 0.7225942760646547, 0.7226865714720575, 0.7226852221659515, 0.7227158347094574, 0.7226957841042854, 0.7227456918153607, 0.7228337985921378, 0.7229039226762725, 0.7229365842961509, 0.7228553731909019, 0.7229485259372507, 0.7228858957978027, 0.7229448775154834, 0.7229422252594283, 0.722933763371285, 0.

 **<font color='teal'> Print the best result by using the '.max' function.</font>**

In [36]:
lgbBO.max

{'target': 0.743738519167891,
 'params': {'lambda_l1': 0.0010131013000669931,
  'lambda_l2': 0.02019798878489275,
  'max_depth': 61.09460156784994,
  'min_child_samples': 3004.2537501258944,
  'min_data_in_leaf': 1903.0329463007906,
  'num_leaves': 1907.551699879095}}

Review the process at each step by using the '.res[0]' function.

In [37]:
lgbBO.res[0]

{'target': 0.7425921058253618,
 'params': {'lambda_l1': 0.007680908239028972,
  'lambda_l2': 0.04067968199112053,
  'max_depth': 60.52039064760403,
  'min_child_samples': 5719.565721298088,
  'min_data_in_leaf': 1343.1710473427568,
  'num_leaves': 2418.592975616843}}